In [1]:
import pandas as pd
import tqdm
from typing import Optional

import re
import nltk

In [2]:
#articles
!gdown 1BAIEjxgx-Q0hGV9ByLajBd-5Id2NZQE5

Downloading...
From: https://drive.google.com/uc?id=1BAIEjxgx-Q0hGV9ByLajBd-5Id2NZQE5
To: /content/articles.csv
100% 25.2M/25.2M [00:00<00:00, 50.8MB/s]


In [3]:
nltk.download('stopwords')
stopword_set = set(nltk.corpus.stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
articles = pd.read_csv('articles.csv')
articles.head()

,date,title,body,count,url
0,2024-02-23,Зеленский без плана Б и с угрозой паралича эко...,Президент Украины Владимир Зеленский не смог о...,673,/mezhdunarodnaya-panorama/20061391
1,2024-02-23,Как советские летчики впервые долетели напряму...,"24 февраля 1899 года, 125 лет назад, родился в...",179,/infographics/9497
2,2024-02-23,"Провал ""украинизации"" G20 и капризы Запада. За...","Запад стремится наказать всех тех, кто не след...",472,/politika/20062849
3,2024-02-23,Военная операция на Украине. Хроника событий 2...,Госдепартамент США внес в свои санкционные спи...,59,/armiya-i-opk/20054555
4,2024-02-23,Военная операция на Украине. Хроника событий 2...,Минобороны РФ объявило об освобождении населен...,51,/armiya-i-opk/20043925


**отделим тела новостей, оставим только буквы, токенизируем, приведём к нижнему регистру**

In [5]:
corpus = articles.body.values
print(f'len(corpus) = {len(corpus)}, type(corpus) = {type(corpus)}\n')
print(corpus[0], end='\n\n')

TOKEN_PATTERN = re.compile('[а-яА-ЯёЁ]+')


def tokenize(text):
    return re.findall(TOKEN_PATTERN, text.lower())


docs = [tokenize(text) for text in corpus]
print(docs[0])

len(corpus) = 2396, type(corpus) = <class 'numpy.ndarray'>

Президент Украины Владимир Зеленский не смог объяснить, в чем заключается его план Б на тот случай, если Вашингтон не предоставит Киеву новый пакет помощи. У украинских властей нет выбора, проводить или не проводить мобилизацию, так как часть подразделений Вооруженных сил Украины (ВСУ) укомплектованы лишь наполовину. Однако, по словам экспертов, мобилизация уже создала огромный дефицит рабочей силы, а дополнительный призыв сотен тысяч человек приведет к частичному параличу экономики. ТАСС собрал актуальную информацию о событиях на Украине и вокруг нее. Вооруженные силы РФ за прошедшие сутки заняли более выгодные рубежи и позиции и отразили девять атак и контратак ВСУ на авдеевском направлении, три атаки на купянском направлении и три атаки на запорожском направлении, сообщили в Минобороны РФ. Потери противника за сутки на всех направлениях, включая донецкое, южнодонецкое и херсонское, составили до 840 бойцов. Российские средст

**почистим от стоп-слов**

In [6]:
cleared_docs = [[token for token in text if token not in stopword_set] for text in tqdm.tqdm(docs)]

100%|██████████| 2396/2396 [00:00<00:00, 8772.88it/s]


**TRIE**

In [7]:
class TrieNode:
    def __init__(self, label: str, key: Optional[str] = None):
        self.children = [None] * 35     # не 33, потому что 'я' --> 'ѐ' --> 'ё'
        self.is_terminal = False
        self.reverse_index = {}


class Trie:
    def __init__(self):
        self.root = TrieNode(label='')

    def insert(self, key, doc_index, word_index):
        if not key:
            return None

        search_result = self.search(key)

        # если слово есть
        if search_result[0]:
            current_node = search_result[1]

            # если слово есть, но нет его документа --> добавляем к слову новый документ и индекс
            if doc_index not in current_node.reverse_index:
                current_node.reverse_index[doc_index] = [word_index]

            # если слово есть и есть его документ --> добавляем к документу новый индекс
            else:
                current_node.reverse_index[doc_index].append(word_index)
            return None


        # если слова нет --> добавляем слово с его индексом документа и индексом в документе
        current_node = self.root
        for char in key:
            index = ord(char) - ord('а')
            if not current_node.children[index]:
                current_node.children[index] = TrieNode(label=char)
            current_node = current_node.children[index]
        current_node.is_terminal = True
        current_node.key = key
        current_node.reverse_index[doc_index] = [word_index]


    def search(self, key):
        if not key:
            return None

        current_node = self.root

        for char in key:
            index = ord(char) - ord('а')
            if not current_node.children[index]:
                return False, ''
            current_node = current_node.children[index]

        return current_node.is_terminal, current_node

**префиксное дерево на текстах, очищенных от стоп-слов**

In [8]:
tass_trie = Trie()

for i in tqdm.tqdm(range(len(cleared_docs))):
    for j in range(len(cleared_docs[i])):
        tass_trie.insert(cleared_docs[i][j], i, j)

100%|██████████| 2396/2396 [00:12<00:00, 186.13it/s]


**поиск по нескольким словам с выдачей топ-k**

In [9]:
def _search(trie: Trie, *words) -> dict:
    """
    ищет слова в префиксном дереве
    возвращает: {word: TrieNode}
    """
    dict_result = {}
    for word in words:
        word = word.lower()
        result = trie.search(word)
        if result[0]:
            dict_result[word] = result[1]
        else:
            print('there is no such word in the database')
    return dict_result


def search(trie: Trie, *words, k=5) -> None:
    """
    вызывает функцию _search()
    отрезает k релевантных документов, печатает сниппеты из текстов
    """

    dict_result = _search(trie, *words)

    print("################################################")
    print("\n=========================================\n")
    for word in dict_result:
        indices = dict_result[word].reverse_index

        if len(indices) < k:
            k = len(indices)
        sorted_indices = sorted(indices.items(), key=lambda item: -len(item[1]))[:k]

        print(f'the word "{word}" appears in:')
        for key in sorted_indices:
            print(f'TITLE: {articles.title[key[0]]}')
            indices_in_doc = key[1]
            for index in indices_in_doc:
                if 0 <= index + 2 < len(cleared_docs[key[0]]) and 0 <= index - 2 < len(cleared_docs[key[0]]):
                    print(f'SNIPPET: "...{cleared_docs[key[0]][index-2]} {cleared_docs[key[0]][index-1]} {cleared_docs[key[0]][index]} {cleared_docs[key[0]][index+1]} {cleared_docs[key[0]][index+2]}..."')
                elif 0 <= index - 2 < len(cleared_docs[key[0]]):
                    print(f'SNIPPET: "...{cleared_docs[key[0]][index-2]} {cleared_docs[key[0]][index-1]} {cleared_docs[key[0]][index]}..."')
                else:
                    print(f'SNIPPET: "...{cleared_docs[key[0]][index]} {cleared_docs[key[0]][index+1]} {cleared_docs[key[0]][index+2]}..."')
            print("\n=========================================\n")
        print("################################################")

In [10]:
search_result = search(tass_trie, 'лукашенко')

################################################


the word "лукашенко" appears in:
TITLE: Перенимать опыт ЧВК, но держа ухо востро. Что сказал Лукашенко главе Минобороны Белоруссии
SNIPPET: "...белоруссии александр лукашенко встрече министром..."
SNIPPET: "...хрениным словам лукашенко минске напряжением..."
SNIPPET: "...моменты беседы лукашенко хренина словам..."
SNIPPET: "...хренина словам лукашенко это бесценно..."
SNIPPET: "...приводит слова лукашенко агентство белта..."
SNIPPET: "...агентство белта лукашенко рассказал разговоре..."
SNIPPET: "...разговаривать словам лукашенко путин разговоре..."
SNIPPET: "...советском союзе лукашенко удивился отметил..."
SNIPPET: "...белоруссии это лукашенко заметил пусть..."


TITLE: ЧВК "Вагнер" и возможные переговоры по Украине осенью. О чем говорил Лукашенко СМИ
SNIPPET: "...белоруссии александр лукашенко встрече представителями..."
SNIPPET: "...россией словам лукашенко будущее планеты..."
SNIPPET: "...пригожина передал лукашенко дальнейшем раб

**OR / AND**

In [11]:
def search_or_and(trie, *words, mode=['or', 'and'], k=5) -> list:

    dict_result = _search(trie, *words)
    tmp_dict = {}
    for word in dict_result:
        indices = dict_result[word].reverse_index
        tmp_dict[word] = {doc_index for doc_index in indices.keys()}

    doc_indices = set()

    if mode == 'or':
        for value in tmp_dict.values():
            doc_indices = doc_indices.union(value)

    elif mode == 'and':
        for value in tmp_dict.values():
            doc_indices = doc_indices.union(value)
            break
        for value in tmp_dict.values():
            doc_indices = doc_indices.intersection(value)

    k = len(doc_indices) if len(doc_indices) < k else k
    doc_indices = list(doc_indices)[:k]

    print(f'with mode "{mode}" the words {words} appear in:')

    for doc_index in doc_indices:
        print(f'TITLE: {articles.title[doc_index]}')

    return doc_indices

In [12]:
rsoa = search_or_and(tass_trie, 'ученые', 'путин', mode='and')

with mode "and" the words ('ученые', 'путин') appear in:
TITLE: Валерий Фальков: науке нужна современная инфраструктура
TITLE: Мегагранты и независимость от зарубежных технологий. О чем Путин говорил с учеными
TITLE: Геннадий Красников: не люблю супермаркеты!
TITLE: Образовательный обмен: чем России и Африке могут быть выгодны совместные проекты
TITLE: Конференция Сбера по искусственному интеллекту AIJ 2023. Текстовая трансляция второго дня
